In [168]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier # Import Random Forest Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import classification_report, confusion_matrix
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, plot_roc_curve, roc_curve
import numpy as np
import matplotlib.pyplot as plt
from random_forest import *

In [195]:
d = pd.read_csv('apks_multilingual_2022_train.csv')
dt = pd.read_csv('apks_multilingual_2022_test.csv')
def fpr_at_low_fnr(y_pred, y_real, fnr=0.05):
    fpr, tpr, thresholds = roc_curve(y_real, y_pred)
    return fpr[tpr >= (1-fnr)][0]

3237


In [201]:
print(len(d.columns), len(dt.columns))
print(len(set(d.columns) - set(dt.columns)), len(set(dt.columns) - set(d.columns)))

3237 1020
2677 460


In [172]:
X, Y, train_columns, _ = read_data(train_file)
print("X_train length: {};  Pos samples: {}".format(
        X.shape, Y.sum()
))

X_train length: (604, 470);  Pos samples: 296


In [272]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
model = RandomForestClassifier(n_estimators=10, max_depth=3, class_weight={0: 1, 1: 1})
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)
tr_acc = accuracy_score(y_train , model.predict(X_train))
ts_acc = accuracy_score(y_test , y_pred[:, 1]>0.5)

print(f"Train acc: {tr_acc} ---> Test acc: {ts_acc}")
tr_fpr = fpr_at_low_fnr(model.predict_proba(X_train)[:, 1], y_train, fnr=0.05)
ts_fpr = fpr_at_low_fnr(model.predict_proba(X_test)[:, 1], y_test, fnr=0.05)
print(f"fpr_at_low_fnr::: Train: {tr_fpr}, Test: {ts_fpr}")

Train acc: 0.7351485148514851 ---> Test acc: 0.73
fpr_at_low_fnr::: Train: 0.6862745098039216, Test: 0.7115384615384616


In [182]:
y_pred, y_real = model.predict_proba(X_train), y_train
print(y_pred, y_real)
fpr, tpr, thresholds = roc_curve(y_real, y_pred[:, 1])
print(fpr, tpr, thresholds)
fpr[tpr >= (1-0.05)][0]

[[0.66540771 0.33459229]
 [0.38834936 0.61165064]
 [0.60638207 0.39361793]
 [0.43890666 0.56109334]
 [0.46099907 0.53900093]
 [0.68025884 0.31974116]
 [0.63881179 0.36118821]
 [0.68025884 0.31974116]
 [0.36354012 0.63645988]
 [0.38648822 0.61351178]
 [0.53854117 0.46145883]
 [0.39899182 0.60100818]
 [0.53211985 0.46788015]
 [0.50207969 0.49792031]
 [0.54393756 0.45606244]
 [0.45158267 0.54841733]
 [0.17894332 0.82105668]
 [0.57550059 0.42449941]
 [0.65366291 0.34633709]
 [0.35285278 0.64714722]
 [0.34529561 0.65470439]
 [0.59394584 0.40605416]
 [0.21305696 0.78694304]
 [0.47814535 0.52185465]
 [0.37183597 0.62816403]
 [0.33492816 0.66507184]
 [0.68025884 0.31974116]
 [0.61740833 0.38259167]
 [0.58224278 0.41775722]
 [0.56104653 0.43895347]
 [0.52746476 0.47253524]
 [0.26395481 0.73604519]
 [0.63228962 0.36771038]
 [0.37183597 0.62816403]
 [0.535211   0.464789  ]
 [0.53138149 0.46861851]
 [0.46396951 0.53603049]
 [0.42271136 0.57728864]
 [0.46410539 0.53589461]
 [0.58615373 0.41384627]


0.6666666666666666

In [127]:
important_features_dict = {}
for idx, val in enumerate(model.feature_importances_):
    important_features_dict[idx] = val

important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print(f'5 most important features: {X.columns[important_features_list[:15]]}')

5 most important features: Index(['receivers_firebaseinstanceidreceiver', 'arm_properties_mounts',
       'arm_properties_exec', 'wide_properties_mms',
       'services_revocationboundservice', 'providers_firebaseperfprovider',
       'services_myfirebaseinstanceidservice',
       'code_properties_hide_softkeyboard', 'permission_foreground_service',
       'receivers_bootreceiver', 'permission_use_credentials',
       'permission_record_audio', 'permission_query_all_packages',
       'permission_write_external_storage', 'permission_access_gps'],
      dtype='object')


In [189]:
for col in X.columns:
    #c = X[col].corr(Y)
    c = (X[col] & Y).sum()
    if c>100 or col in {'arm_properties_exec', 'arm_properties_exec'}:
        print(col, c)

receivers_firebaseinstanceidreceiver 205
permission_internet 293
permission_write_external_storage 212
permission_access_coarse_location 167
permission_vibrate 151
permission_receive_boot_completed 192
code_properties_encryption 182
code_properties_fingerprint 176
code_properties_hide_softkeyboard 148
code_properties_http 149
code_properties_json 221
code_properties_manufacturer 208
code_properties_post 231
code_properties_user_agent 210
wide_properties_mms 174
wide_properties_play_services 276
receivers_forcestoprunnable$broadcastreceiver 102
permission_read_external_storage 147
permission_billing 104
permission_foreground_service 166
providers_fileprovider 134
code_properties_accessibility_service 122
code_properties_android_id 114
code_properties_check_permission 104
code_properties_get_imei 120
code_properties_get_sim_operator 119
code_properties_gps 149
code_properties_package_sig 136
code_properties_stacktrace 150
code_properties_uuid 178
code_properties_zip 115
arm_properties_ex

# Score of NLP classifier

In [202]:
import pandas as pd
import sqlite3

con = sqlite3.connect("/mnt/a/research_db/IPV_spyware/Final_DB_(2020_and_2022_crawls)/Merged_&_Translated_with_score_(No_Terms).db")

In [206]:
d = pd.read_sql("select * from android_apps where score>0.4", con).set_index('appId')
import json
import ast
def parse(x):
    try: 
        return ast.literal_eval(x) 
    except: 
        return []
d['similar'] = d.similar.apply(parse)

In [240]:
sim_apps = np.concatenate(d.similar.to_list())
sim_scores = d.loc[set(sim_apps) & set(d.index)].ml_score.to_dict()

In [241]:
d['sim_scores'] = d.similar.apply(lambda x: np.mean([sim_scores[y] for y in x if y in sim_scores]))

/u/c/h/chatterjee/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [274]:
import sqlite3
con = sqlite3.connect("/mnt/a/research_db/IPV_spyware/all_crawls.db")

In [275]:
d2020 = pd.read_csv('Full_app_list_2020.csv')

In [281]:
d = pd.read_sql('select * from android_apps where crawl_year="2018"', con)

In [292]:
d2020.columns, d.columns

(Index(['appId', 'genreId', 'summary', 'title', 'description', 'AppWebsite',
        'permissions', 'y', 'ml_score', 'relevant', 'Reviewer1 Comment',
        'Still Available (20220228)'],
       dtype='object'),
 Index(['', 'appId', 'title', 'description', 'ml_score', 'detected', 'LANG',
        'COUNTRY', 'crawl_year', 'relevant'],
       dtype='object'))

In [299]:
dd = d[['appId', 'title', 'description', 'ml_score', 'detected', 'LANG',
        'COUNTRY', 'crawl_year', 'relevant']]

In [302]:
18177/27585

0.6589450788471996